<a href="https://colab.research.google.com/github/fzanart/GHDomains/blob/main/Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
%%capture
!pip install -U sentence-transformers
!pip install flaml

In [48]:
from sentence_transformers import SentenceTransformer
from flaml import AutoML
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.metrics import classification_report, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import markdown
import lxml
from bs4 import BeautifulSoup
import re
import time
import pickle
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [49]:
#Imports:

# Clasification model
with open('/content/drive/MyDrive/GHDomains/Resources/automl.pkl', 'rb') as fp:
  automl = pickle.load(fp)
# Label encoder
with open('/content/drive/MyDrive/GHDomains/Resources/label_encoder.pkl', 'rb') as fp:
  le = pickle.load(fp)
# Num scaler
with open('/content/drive/MyDrive/GHDomains/Resources/num_scaler.pkl', 'rb') as fp:
  scaler = pickle.load(fp)
# Column names
with open('/content/drive/MyDrive/GHDomains/Resources/column_names.pkl', 'rb') as fp:
  column_names = pickle.load(fp)

In [50]:
new_popular_df = pd.read_csv('/content/drive/MyDrive/GHDomains/New_popular_download/all_repos.csv')
new_popular_df = new_popular_df.rename(columns={"README": "Readme"}) 
new_popular_df

,Name,Description,Readme,Labels,Contributors,Languages,Topics,Contents,Licence,Stars,Forks,Releases,Workflows
0,vuejs/devtools,⚙️ Browser devtools extension for debugging Vu...,# vue-devtools\n\n![screenshot](./media/screen...,"['accepted proposition', 'bug', 'cannot reprod...","['Akryum', 'yyx990803', 'posva', 'bartlomieju'...","['TypeScript', 'Vue', 'JavaScript', 'HTML', 'S...",NaN,"['.browserslistrc', '.circleci', '.eslintrc.js...",MIT License,22091,3684,73,['Create Release']
1,futurice/android-best-practices,"Do's and Don'ts for Android development, by Fu...",# Best practices in Android development\n\nAvo...,"['2018', 'answered', 'bug', 'discussion', 'dup...","['staltz', 'peter-tackage', 'minsoopark', 'and...",NaN,"['best-practices', 'android-development', 'and...","['LICENSE', 'README.md', 'translations']",Other,19943,3305,0,NaN
2,microsoft/Web-Dev-For-Beginners,"24 Lessons, 12 Weeks, Get Started as a Web Dev...",[![GitHub license](https://img.shields.io/gith...,"['bug', 'dependencies', 'documentation', 'dupl...","['jlooper', 'ManuSquall', 'San1ay', 'silversky...","['JavaScript', 'HTML', 'CSS', 'Vue']","['javascript', 'curriculum', 'html', 'css', 'e...","['.github', '.gitignore', '.nojekyll', '1-gett...",MIT License,44328,6279,0,"['Azure Static Web Apps CI/CD', 'Lock closed i..."
3,airbnb/react-sketchapp,render React components to Sketch ⚛️💎,"<div align=""center"">\n <img alt=""react-sketch...","['awaiting-review', 'bug', 'chore', 'discuss',...","['jongold', 'mathieudutour', 'macintoshhelper'...","['TypeScript', 'JavaScript']","['react-sketchapp', 'react', 'sketch', 'sketch...","['.bookignore', '.editorconfig', '.github', '....",MIT License,14874,864,28,NaN
4,eugeneyan/applied-ml,📚 Papers & tech blogs by companies sharing the...,"# applied-ml\nCurated papers, articles, and bl...","['bug', 'documentation', 'duplicate', 'enhance...","['eugeneyan', 'shreyansh26', 'chmnsk', 'nilesh...",NaN,"['applied-machine-learning', 'production', 'ap...","['CONTRIBUTING.md', 'LICENSE', 'README.md']",MIT License,19001,2615,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,railsware/upterm,A terminal emulator for the 21st century.,[![Join the chat at https://gitter.im/railswar...,"['ANSI', 'Autocompletion', 'Blocked', 'Bug: Co...","['vlad-shatskyi', 'drew-gross', 'ShockOne', 'g...","['TypeScript', 'HTML', 'Dockerfile']","['tty', 'terminal', 'terminal-emulators', 'con...","['.dockerignore', '.gitignore', '.node-version...",MIT License,19415,657,187,NaN
437,matryer/xbar,Put the output from any script or program into...,[![](xbarapp.com/public/img/xbar-menu-preview....,"['! invalid', '? question', '→? future maybe',...","['matryer', 'iosdeveloper', 'leaanthony', 'muh...","['Go', 'Objective-C', 'HTML', 'Svelte', 'JavaS...","['mac', 'macosx', 'bitbar', 'menubar', 'script...","['.github', '.gitignore', '.vscode', 'LICENSE....",MIT License,15908,615,69,NaN
438,thedaviddias/Front-End-Checklist,🗂 The perfect Front-End Checklist for modern w...,"<h1 align=""center"">\n<br>\n <img src=""https:/...","['#Status: Add into the app', '#Status: Awaiti...","['thedaviddias', 'greenkeeper[bot]', 'jochenki...",NaN,"['frontend', 'front-end-development', 'front-e...","['.editorconfig', '.github', '.gitignore', 'CH...",Creative Commons Zero v1.0 Universal,58604,4783,1,['pages-build-deployment']
439,electronicarts/CnC_Remastered_Collection,NaN,# CnC_Remastered_Collection,"['bug', 'documentation', 'duplicate', 'enhance...","['PG-SteveT', 'bmattea', 'rparolin']","['C++', 'C#', 'C', 'Assembly', 'Python', 'Obje...",NaN,"['CnCRemastered.sln', 'CnCTDRAMapEditor.sln', ...",Other,15990,4274,0,NaN


In [51]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [52]:
def join_words_from_lists(text, join=False):
  text = str(text)
  if text == 'nan':
    return ''
  else:
    text = text[1:-1]
    text = text.replace('\'','')
    text = text.split(',')
    if join:
      text = ' '.join([word.strip() for word in text])
    return text

def clean_htmltags(text):
    html = markdown.markdown(text)                                # convert the code to html
    soup = BeautifulSoup(html, "lxml")                            # html.parser #XML stands for "eXtensible Markup Language",
    
    [s.extract() for s in soup(['iframe', 'script'])]             # eliminate html tags 
    stripped_text = soup.get_text()                               # get the rest of the text
    stripped_text = re.sub(r'[\r|\n|\r\n]+', ' ', stripped_text)  # delete remaining tags
    text = stripped_text                                          # voila!
                                    
    return text

def vectorize_categorical(df_Serie, prefix):
    lp_index = df_Serie.index

    vect = CountVectorizer(tokenizer=join_words_from_lists)
    lp_Serie = vect.fit_transform(df_Serie.astype(str))

    lp_Serie = pd.DataFrame(lp_Serie.toarray(), columns=vect.get_feature_names_out(), index=lp_index).add_prefix(prefix)
    lp_Serie = lp_Serie.astype(pd.SparseDtype(np.int64))
    return lp_Serie

In [53]:
def fit_transform(df, selected_features):

  # Apply text pre-processing
  df['Labels'] = df['Labels'].apply(lambda x: join_words_from_lists(x, join=True))
  df['Readme'] = df['Readme'].astype(str)
  df['Readme'] = pd.Series([clean_htmltags(x) for x in df['Readme']]) #df['Readme'].apply(lambda x: clean_htmltags(x)) #
  df['Readme'] = df['Readme'].apply(lambda x: re.sub('\s{2,}', " ", x))
  df['Description'] = df['Description'].astype(str)
  df['Description'] = df['Description'].replace('nan','')
  df['Text_data']   = df['Description'] + df['Readme'] + df['Labels']
  transformed_df = model.encode(df['Text_data'].to_list())

  # Vectorise categorical features
  cn_vect = vectorize_categorical(df['Contributors'], 'cn: ')
  pl_vect = vectorize_categorical(df['Languages'], 'pl: ')
  tp_vect = vectorize_categorical(df['Topics'], 'tp: ')
  ct_vect = vectorize_categorical(df['Contents'], 'ct: ')
  li_vect = vectorize_categorical(df['Licence'], 'li: ')

  # transform numbers of stars, forks y releases
  num_vectors = df[['Stars','Forks', 'Releases']].astype(np.float64)
  num_columns = num_vectors.columns
  num_index = num_vectors.index
  num_vectors = scaler.transform(num_vectors)
  num_vectors = pd.DataFrame(num_vectors, columns=num_columns, index=num_index)

  # Concat the dataframe and filter the columns to plug into the model
  transformed_df = pd.DataFrame(transformed_df, index=df.index).add_prefix('Text ')
  transformed_df = pd.concat([transformed_df, cn_vect, pl_vect, tp_vect, ct_vect, li_vect, num_vectors], axis=1)
  transformed_df = transformed_df.reindex(columns=column_names, fill_value=0)

  return transformed_df

In [54]:
transformed_df = fit_transform(df=new_popular_df, selected_features=column_names)
prediction = automl.predict(transformed_df.to_numpy())

In [55]:
le.inverse_transform(prediction)

array(['Web libraries and frameworks', 'Documentation', 'Documentation',
       'Web libraries and frameworks', 'Documentation', 'Software tools',
       'Software tools', 'Web libraries and frameworks', 'Documentation',
       'Software tools', 'Documentation', 'Documentation',
       'Documentation', 'Software tools', 'Web libraries and frameworks',
       'Web libraries and frameworks', 'Application & System software',
       'Documentation', 'Application & System software',
       'Non-web libraries and frameworks', 'Web libraries and frameworks',
       'Documentation', 'Web libraries and frameworks', 'Documentation',
       'Software tools', 'Documentation', 'Application & System software',
       'Software tools', 'Documentation', 'Application & System software',
       'Software tools', 'Software tools', 'Web libraries and frameworks',
       'Software tools', 'Web libraries and frameworks', 'Documentation',
       'Software tools', 'Application & System software',
       'Appl

In [58]:
new_popular_df['Predicted_labels'] = le.inverse_transform(prediction)

In [59]:
new_popular_df

,Name,Description,Readme,Labels,Contributors,Languages,Topics,Contents,Licence,Stars,Forks,Releases,Workflows,Text_data,Predicted_labels
0,vuejs/devtools,⚙️ Browser devtools extension for debugging Vu...,vue-devtools Documentation Install the extensi...,accepted proposition bug cannot reproduce cont...,"['Akryum', 'yyx990803', 'posva', 'bartlomieju'...","['TypeScript', 'Vue', 'JavaScript', 'HTML', 'S...",NaN,"['.browserslistrc', '.circleci', '.eslintrc.js...",MIT License,22091,3684,73,['Create Release'],⚙️ Browser devtools extension for debugging Vu...,Web libraries and frameworks
1,futurice/android-best-practices,"Do's and Don'ts for Android development, by Fu...",Best practices in Android development Avoid re...,2018 answered bug discussion duplicate enhance...,"['staltz', 'peter-tackage', 'minsoopark', 'and...",NaN,"['best-practices', 'android-development', 'and...","['LICENSE', 'README.md', 'translations']",Other,19943,3305,0,NaN,"Do's and Don'ts for Android development, by Fu...",Documentation
2,microsoft/Web-Dev-For-Beginners,"24 Lessons, 12 Weeks, Get Started as a Web Dev...",Web Development for Beginners - A Curriculum ...,bug dependencies documentation duplicate enhan...,"['jlooper', 'ManuSquall', 'San1ay', 'silversky...","['JavaScript', 'HTML', 'CSS', 'Vue']","['javascript', 'curriculum', 'html', 'css', 'e...","['.github', '.gitignore', '.nojekyll', '1-gett...",MIT License,44328,6279,0,"['Azure Static Web Apps CI/CD', 'Lock closed i...","24 Lessons, 12 Weeks, Get Started as a Web Dev...",Documentation
3,airbnb/react-sketchapp,render React components to Sketch ⚛️💎,render React components to Sketch; tailor-mad...,awaiting-review bug chore discuss docs duplica...,"['jongold', 'mathieudutour', 'macintoshhelper'...","['TypeScript', 'JavaScript']","['react-sketchapp', 'react', 'sketch', 'sketch...","['.bookignore', '.editorconfig', '.github', '....",MIT License,14874,864,28,NaN,render React components to Sketch ⚛️💎 render R...,Web libraries and frameworks
4,eugeneyan/applied-ml,📚 Papers & tech blogs by companies sharing the...,"applied-ml Curated papers, articles, and blogs...",bug documentation duplicate enhancement good f...,"['eugeneyan', 'shreyansh26', 'chmnsk', 'nilesh...",NaN,"['applied-machine-learning', 'production', 'ap...","['CONTRIBUTING.md', 'LICENSE', 'README.md']",MIT License,19001,2615,0,NaN,📚 Papers & tech blogs by companies sharing the...,Documentation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,railsware/upterm,A terminal emulator for the 21st century.,Deprecated Upterm is deprecated. Project had ...,ANSI Autocompletion Blocked Bug: Confirmed Bug...,"['vlad-shatskyi', 'drew-gross', 'ShockOne', 'g...","['TypeScript', 'HTML', 'Dockerfile']","['tty', 'terminal', 'terminal-emulators', 'con...","['.dockerignore', '.gitignore', '.node-version...",MIT License,19415,657,187,NaN,A terminal emulator for the 21st century. Depr...,Software tools
437,matryer/xbar,Put the output from any script or program into...,Welcome to xbar xbar (the BitBar reboot) lets...,! invalid ? question →? future maybe ♡ todo bu...,"['matryer', 'iosdeveloper', 'leaanthony', 'muh...","['Go', 'Objective-C', 'HTML', 'Svelte', 'JavaS...","['mac', 'macosx', 'bitbar', 'menubar', 'script...","['.github', '.gitignore', '.vscode', 'LICENSE....",MIT License,15908,615,69,NaN,Put the output from any script or program into...,Software tools
438,thedaviddias/Front-End-Checklist,🗂 The perfect Front-End Checklist for modern w...,Front-End Checklist 🚨 Currently working on a ...,#Status: Add into the app #Status: Awaiting re...,"['thedaviddias', 'greenkeeper[bot]', 'jochenki...",NaN,"['frontend', 'front-end-development', 'front-e...","['.editorconfig', '.github', '.gitignore', 'CH...",Creative Commons Zero v1.0 Universal,58604,4783,1,['pages-build-deployment'],🗂 The perfect Front-End Checklist for modern w...,Documentation
439,electronicarts/CnC_Remastered_Collection,,CnC_Remastered_Collection,bug documentation duplicate enhancement g

In [60]:
new_popular_df.to_csv('/content/drive/MyDrive/GHDomains/Resources/new_popular_df_predicted.csv', index=False)

In [56]:
# TODO: fix clasification results. for target names use le.clases_
# Report for less popular, anotated dataset.
#print(classification_report(lp_true, lp_pred, target_names=le.classes_))